In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fmin_cobyla

## (a)

In [ ]:


# Given payoff matrix A and prices p
A = np.array([
    [5, 30, 5],
    [5, 10, 0],
    [5, 0, 10]
])

P = np.array([1, 2, 1])

#Compute basic prices for matrix of payoffs and prices
def compute_basic(A,P):
    num_states = len(A)
    num_assets = len(P)
    subA = A[:,0:num_states]
    subP = P[0:num_states]
    pb = np.linalg.inv(np.transpose(subA)) @ subP
    return pb

pb = compute_basic(A,P)
print(pb)

[0.04 0.08 0.08]


## (b)

In [10]:
def check_asset_mispricing(A_new, p_new, arrow_prices):

    # Compute theoretical price
    p_star = np.dot(A_new, arrow_prices)
    # Determine mispricing
    if p_star > p_new:
        status = "The asset is too cheap (underpriced)."
        arbitrage_profit = p_star - p_new
    elif p_star < p_new:
        status = "The asset is too expensive (overpriced)."
        arbitrage_profit = p_new - p_star
    else:
        status = "The asset is correctly priced."
        arbitrage_profit = 0.0
    return p_star, status, arbitrage_profit

# Given new asset A3 and price p3
A3 = np.array([2, 2, 3])
p3 = 4

# Check mispricing
p_star, status, arbitrage_profit = check_asset_mispricing(A3, p3, pb)

print(status)
print(f"Sell 1 unit of asset 3 at ${p3} and buy the portfolio replication (2 unit of p̄0, 2 units of p̄1, 3 units of p̄2) for ${p_star:.2f}")
print(f"Arbitrage profit per unit: {arbitrage_profit:.2f}")


The asset is too expensive (overpriced).
Sell 1 unit of asset 3 at $4 and buy the portfolio replication (2 unit of p̄0, 2 units of p̄1, 3 units of p̄2) for $0.48
Arbitrage profit per unit: 3.52


## (c)

In [15]:

# %%
"Given basic prices, compute optimal consumption"
# Negative of the utility function
prob_1 = 0.2
prob_2 = 0.3
prob_3 = 0.5
theta = 4
m = 2000

def U(x):
    return -(prob_1*(x[0])**(1-theta)/(1-theta)+prob_2*x[1]**(1-theta)/(1-theta)+prob_3*x[2]**(1-theta)/(1-theta))

# Constraints
def budget(x):
    return m - x[0]*pb[0] - x[1]*pb[1]-x[2]*pb[2]

def x_1_positive(x):
    return x[0]

def x_2_positive(x):
    return x[1]

def x_3_positive(x):
    return x[2]

solution = fmin_cobyla(U, [10000,10000,10000], [budget, x_1_positive, x_2_positive, x_3_positive], rhoend = 1e-7)

print(solution)
print("Total cost:", sum(pb*solution))

[10048.32808784  9351.02283908 10624.81311699]
Total cost: 2000.0


## (d)

In [17]:
# Solve for omega
omega = np.linalg.solve(A, solution)

print("\nOptimal Portfolio omega_n (Quantities of Assets):")
for n in range(len(omega)):
    print(f"omega_{n} = {omega[n]:.4f}")

# Verify the portfolio generates the desired consumption x_t
x_t_check = A.dot(omega)
print("\nVerification: A * omega = x_t (Should match the computed x_t)")
for t in range(len(x_t_check)):
    print(f"State {t}: x_t = {solution[t]:.4f}, A * omega = {x_t_check[t]:.4f}")

# Verify the budget constraint with asset prices
total_portfolio_cost = np.dot(p, omega)
print(f"\nTotal portfolio cost: {total_portfolio_cost:.2f} (Jake's Wealth m = {m})")




Optimal Portfolio omega_n (Quantities of Assets):
omega_0 = 1865.3718
omega_1 = 2.4164
omega_2 = 129.7954

Verification: A * omega = x_t (Should match the computed x_t)
State 0: x_t = 10048.3281, A * omega = 10048.3281
State 1: x_t = 9351.0228, A * omega = 9351.0228
State 2: x_t = 10624.8131, A * omega = 10624.8131

Total portfolio cost: 2000.00 (Jake's Wealth m = 2000)
